# Integrate Brochure Builder with Gradio UI

## Imports & Setup

In [11]:
import os
import requests
import json
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [3]:
# Load env variables, print key prefixes to help w debugging

load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')
deepseek_key = os.getenv('DEEPSEEK_API_KEY')

if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI key not set.")

if anthropic_key:
    print(f"Anthropic API Key exists and begins {anthropic_key[:7]}")
else:
    print("Anthropic key not set.")

if google_key:
    print(f"Google API Key exists and begins {google_key[:8]}")
else:
    print("Google API Key not set.")

if deepseek_key:
    print(f"DeepSeek API Key exists and begins {deepseek_key[:5]}")
else:
    print("DeepSeek key not set.")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBo
DeepSeek API Key exists and begins sk-18


In [4]:
# Connect to OpenAI, Anthropic, Google

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Brochure Builder

### Class for Webpage w links

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """Utility class to represent website that we have scraped, now w links"""

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
# Just to test it's working
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

### Determine Relevant Links

#### Prompts

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage." \
"You are able to decide which of the links would be most relevant to include in a brochure about the company," \
"such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.You are able to decide which of the links would be most relevant to include in a brochure about the company,such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
# Build User Prompts

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company. "
    user_prompt += "Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links. \n"
    user_prompt += "Links (some might be relative links): \n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company. Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links. 
Links (some might be relative links): 
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/2

#### GPT - Get Links

In [12]:
def get_links_gpt(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                "role": "system",
                "content": link_system_prompt
            },
            {
                "role": "user",
                "content": get_links_user_prompt(website)
            }
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

#### Claude - Get Links

In [28]:
def get_links_claude(url):
    website = Website(url)
    response = claude.messages.create(
        model='claude-3-7-sonnet-latest',
        max_tokens=1000,
        system=link_system_prompt,
        messages=[
            {
                "role": "user",
                "content": get_links_user_prompt(website)
            }
        ]
    )
    result = response.content[0].text
    return result

In [13]:
huggingface = "https://huggingface.co"
hug = Website(huggingface)
hug.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/HiDream-ai/HiDream-I1-Full',
 '/agentica-org/DeepCoder-14B-Preview',
 '/moonshotai/Kimi-VL-A3B-Thinking',
 '/moonshotai/Kimi-VL-A3B-Instruct',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/Efficient-Large-Model/SanaSprint',
 '/spaces/HiDream-ai/HiDream-I1-Dev',
 '/spaces',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/openai/mrcr',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets/divaroffical/real_estate_ads',
 '/datasets/openai/graphwalks',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micro

In [14]:
get_links_gpt(huggingface)

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

In [29]:
get_links_claude(huggingface)

'```json\n{\n  "links": [\n    {"type": "home page", "url": "https://huggingface.co/"},\n    {"type": "about page", "url": "https://huggingface.co/huggingface"},\n    {"type": "brand page", "url": "https://huggingface.co/brand"},\n    {"type": "products page", "url": "https://huggingface.co/models"},\n    {"type": "products page", "url": "https://huggingface.co/datasets"},\n    {"type": "products page", "url": "https://huggingface.co/spaces"},\n    {"type": "enterprise page", "url": "https://huggingface.co/enterprise"},\n    {"type": "pricing page", "url": "https://huggingface.co/pricing"},\n    {"type": "documentation page", "url": "https://huggingface.co/docs"},\n    {"type": "blog page", "url": "https://huggingface.co/blog"},\n    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},\n    {"type": "community page", "url": "https://discuss.huggingface.co"},\n    {"type": "github page", "url": "https://github.com/huggingface"},\n    {"type": "twitter page", "url"

In [32]:
# Build User Prompt using Claude-generated prompt to improve output above

def get_links_user_prompt_claude(website):
    user_prompt = f"Given the following links found on the company website of {website.url}, \
    identify which ones would be most relevant to include in a company brochure."
    user_prompt += "Focus on links like 'About Us', 'Company Information', 'Our Mission', 'Careers', 'Jobs', etc."
    user_prompt += "Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, or email links. \n"
    user_prompt += "Links (some might be relative links): \n"
    user_prompt += "\n".join(website.links)
    user_prompt += "\n\nFor each relevant link, specify its type (e.g., 'about page', 'careers page', etc.)\n"
    user_prompt += "Return your answer in this exact JSON format:\n"
    user_prompt += """{{
        "links": [
            {{"type": "about page", "url": "https://full.url/goes/here/about"}},
            {{"type": "careers page", "url": "https://another.full.url/careers"}},
            ...
        ]
    }}
    Only include the JSON in your response, and nothing else."""
    return user_prompt

In [35]:
print(get_links_user_prompt_claude(hug))

Given the following links found on the company website of https://huggingface.co,     identify which ones would be most relevant to include in a company brochure.Focus on links like 'About Us', 'Company Information', 'Our Mission', 'Careers', 'Jobs', etc.Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, or email links. 
Links (some might be relative links): 
/
/models
/datasets
/spaces
/posts
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/HiDream-ai/HiDream-I1-Full
/agentica-org/DeepCoder-14B-Preview
/moonshotai/Kimi-VL-A3B-Thinking
/moonshotai/Kimi-VL-A3B-Instruct
/deepseek-ai/DeepSeek-V3-0324
/models
/spaces/enzostvs/deepsite
/spaces/jamesliu1217/EasyControl_Ghibli
/spaces/bytedance-research/UNO-FLUX
/spaces/Efficient-Large-Model/SanaSprint
/spaces/HiDream-ai/HiDream-I1-Dev
/spaces
/datasets/nvidia/OpenCodeReasoning
/datasets/openai/mrcr
/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset
/datasets/divaroffical/real_estate_ads
/data

In [33]:
# Reworked get_links_claude function using Claude input

def claude_says_get_links(url):
    website = Website(url)
    response = claude.messages.create(
        model='claude-3-7-sonnet-latest',
        max_tokens=1000,
        system=link_system_prompt,
        messages=[
            {
                "role": "user",
                "content": get_links_user_prompt_claude(website)
            }
        ]
    )
    result = response.content[0].text
    return result

In [34]:
claude_says_get_links(huggingface)

'{\n    "links": [\n        {"type": "about page", "url": "https://huggingface.co/huggingface"},\n        {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},\n        {"type": "company information", "url": "https://huggingface.co/enterprise"},\n        {"type": "brand page", "url": "https://huggingface.co/brand"}\n    ]\n}'

In [36]:
import re

### Claude JSON Output

In [37]:
# Reworking original Claude function that uses same user prompt as gpt, tweaking to output instead

def get_json_links_claude(url):
    website = Website(url)
    response = claude.messages.create(
        model='claude-3-7-sonnet-latest',
        max_tokens=1000,
        system=link_system_prompt,
        messages=[
            {
                "role": "user",
                "content": get_links_user_prompt(website)
            }
        ]
    )
    raw_data = response.content[0].text
    
    # Extract JSON portion from Claude's response
    try:
        # Try to parse directly if response is clean JSON
        if raw_data.strip().startswith("{"):
            json_data = json.loads(raw_data)
        else:
            # If there's extra text, try to extract just the JSON block
            json_match = re.search(r'(\{.*\})', raw_data, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                json_data = json.loads(json_str)
            else:
                raise ValueError("Could not extract JSON from Claude's Response")
            
        # Validate expected structure
        if "links" not in json_data or not isinstance(json_data["links"], list):
            raise ValueError("Invalid response format from Claude API")
        

        return json_data
    
    except json.JSONDecodeError:
        raise ValueError(f"Could not parse Claude's response as JSON: {raw_data}")

In [38]:
print(get_json_links_claude(huggingface))

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand information', 'url': 'https://huggingface.co/brand'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingfa

### Make the Brochure

#### GPT: Get Contents from All Links

In [15]:
# GPT

def get_all_details_gpt(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    # get relevant links from openai
    links = get_links_gpt(url)
    print("Found links: ", links)
    # get contents from each link from relevant links
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details_gpt(huggingface))

Found links:  {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
HiDream-ai/HiDream-I1-Full
Updated
about 6 hours ago
•
18.3k
•
489
agentica-org/DeepCoder-14B-Preview
Updated
7 days ago
•
14.9k
•
548
moonshotai/Kimi-VL-A3B-Thinking
Updated
2 days ago
•
13.8k
•
325
moonshotai/Kimi-VL-A3B-Instruct
Updated
1 day ago
•
12.9k
•
164
deepseek-ai/DeepSeek-V3-0324
Updated
21 days ago
•
226k
•
2.62k
Browse 1M+ models
Spaces
Running

#### Claude: Get Contents from All Links

In [30]:
# Claude

def get_all_details_claude(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    # get relevant links from claude
    links = get_links_claude(url)
    print("Found links: ", links)
    # get contents from each link from relevant links
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [31]:
print(get_all_details_claude(huggingface))

Found links:  ```json
{
  "links": [
    {"type": "home page", "url": "https://huggingface.co/"},
    {"type": "models page", "url": "https://huggingface.co/models"},
    {"type": "datasets page", "url": "https://huggingface.co/datasets"},
    {"type": "spaces page", "url": "https://huggingface.co/spaces"},
    {"type": "blog/posts page", "url": "https://huggingface.co/posts"},
    {"type": "documentation page", "url": "https://huggingface.co/docs"},
    {"type": "enterprise page", "url": "https://huggingface.co/enterprise"},
    {"type": "pricing page", "url": "https://huggingface.co/pricing"},
    {"type": "company brand page", "url": "https://huggingface.co/brand"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "learning resources", "url": "https://huggingface.co/learn"},
    {"type": "community forum", "url": "https://discuss.huggingface.co"},
    {"type": "status page", "url": "https://status.huggingface.co/"},
    {"type": "github pa

TypeError: string indices must be integers, not 'str'

In [39]:
# With reworked get links function

# Claude

def get_all_details_now_claude(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    # get relevant links from claude
    links = get_json_links_claude(url)
    print("Found links: ", links)
    # get contents from each link from relevant links
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [40]:
print(get_all_details_now_claude(huggingface))

Found links:  {'links': [{'type': 'main page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and appl

#### Prompts

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
 and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
 Include details of company culture, customers and careers/jobs if you have the information."

#### GPT: Brochure User Prompt

In [19]:
def get_gpt_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += "Here are the contents of its landing page and other relevant pages;"
    user_prompt += "use this information to build a short brochure of the company in markdown. \n"
    user_prompt += get_all_details_gpt(url)
    user_prompt += user_prompt[:5_000] # Truncate of more than 5,000 characters
    return user_prompt

In [20]:
get_gpt_brochure_user_prompt("HuggingFace", huggingface)

Found links:  {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;use this information to build a short brochure of the company in markdown. \nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nHiDream-ai/HiDream-I1-Full\nUpdated\nabout 6 hours ago\n•\n18.3k\n•\n489\nagentica-org/DeepCoder-14B-Preview\nUpdated\n7 days ago\n•\n14.9k\n•\n548\nmoonshotai/Kimi-VL-A3B-Thinking\nUpdated\n2 days ago\n•\n13.8k\n•\n325\nmoonshotai/Kimi-VL-A3B-Instruct\nUpdated\n1 day ago\n•\n12.9k\n•\n164\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n21 days ago\n•\n226k\n•\n2.62k\nBrowse 1M+ models\nSpaces\nRunnin

#### Claude: Brochure User Prompt

In [41]:
def get_claude_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += "Here are the contents of its landing page and other relevant pages;"
    user_prompt += "use this information to build a short brochure of the company in markdown. \n"
    user_prompt += get_all_details_now_claude(url)
    user_prompt += user_prompt[:5_000] # Truncate of more than 5,000 characters
    return user_prompt

In [42]:
get_claude_brochure_user_prompt("HuggingFace", huggingface)

Found links:  {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;use this information to build a short brochure of the company in markdown. \nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nHiDream-ai/HiDream-I1-Full\nUpdated\nabout 7 hours ago\n•\n18.3k\n•\n489\nagentica-org/DeepCoder-14B-Preview\nUpdated\n7 days ago\n•\n14.9k\n•\n549\nmoonshotai/Kimi-VL-A3B-Thinking\nUpdated\n2 days ago\n•\n13.8k\n•\n325\nmoonshotai/Kimi-VL-A3B-Instruct\nUpdated\n1 day ago\n•\n12.9k\n•\n165\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n21 days ago\n•\n226k\n•\n2.62k\nBrowse 1M+ models\nSpaces\nRunnin

#### Stream GPT Brochure

In [24]:
def gpt_stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": get_gpt_brochure_user_prompt(company_name, url)
            }
        ],
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

#### Stream Claude Brochure

In [43]:
def claude_stream_brochure(company_name, url):
    result = claude.messages.stream(
        model='claude-3-7-sonnet-latest',
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {
                "role": "user",
                "content": get_claude_brochure_user_prompt(company_name, url)
            }
        ]
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ''
            yield response

## Add Gradio UI

### Simple - Single Model

#### GPT: Single Model

In [27]:
view = gr.Interface(
    fn=gpt_stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"), gr.Textbox(label="Company URL:")],
    outputs=[gr.Markdown(label="Your New Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Found links:  {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


#### Claude: Single Model

In [44]:
view = gr.Interface(
    fn=claude_stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"), gr.Textbox(label="Company URL:")],
    outputs=[gr.Markdown(label="Your New Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


Found links:  {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github', 'url': 'https://github.com/huggingfa

## Choose Model

In [45]:
def stream_model(company_name, url, model):
    if model == "GPT":
        result = gpt_stream_brochure(company_name, url)
    elif model == "Claude":
        result = claude_stream_brochure(company_name, url)
    else:
        raise ValueError("Unknown Model: Please Choose GPT or Claude")
    yield from result

In [46]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Company Name:"), gr.Textbox(label="Company URL:"), gr.Dropdown(["GPT", "Claude"], label="Select model:", value="GPT")],
    outputs=[gr.Markdown(label="Your New Brochure")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Found links:  {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
